In [1]:
import pandas as pd
import re
import json 
import pickle

In [2]:
scraping_path = '../../../local_files/Projects/Instagram_Likes_Local/Scraping_Data_files/'
data_path = '../../../local_files/Projects/Instagram_Likes_Local/data/'

In [3]:
with open(scraping_path+"randomUsers.txt", "rt") as f:
    start_urls = [url.strip() for url in f.readlines()]
#     print(start_urls)

In [4]:
df_urls = pd.DataFrame(start_urls)

In [5]:
df_urls = df_urls.drop_duplicates()

In [6]:
df_urls.to_csv(r'randUsernames.txt', header=None, index=None, sep=' ', mode='a')

## Get Usernames

In [7]:
goodPhotos = pd.read_csv(scraping_path+'goodPhotos.csv')
randPhotos = pd.read_csv(scraping_path+'randUsers1Data.csv')
goodPhotos['pro'] = 1
randPhotos['pro'] = 0
meta_data = goodPhotos.append(randPhotos)
meta_data.tail(2)

,caption,comments,image_urls,images,likes,pro,user_url,username
121426,#bonplan #soldes #pimkie @pimkie ____________...,10.0,http://scontent.cdninstagram.com/t51.2885-15/s...,[{'path': 'full/4432f9d9d6e90ab1147789a67a6694...,192.0,0,http://www.imgrum.org/user/idee2look/300983387,Stephanie (@idee2look)
121427,Look du jour #idee2look avec manteau #promod 2...,13.0,http://scontent.cdninstagram.com/t51.2885-15/s...,[{'path': 'full/259d398f595d5e19e5f291333b850c...,145.0,0,http://www.imgrum.org/user/idee2look/300983387,Stephanie (@idee2look)


In [8]:
meta_data = meta_data[(meta_data['username'].notnull())]

In [9]:
def getUsername(row):
    a = re.search(r'\(@[^)]*\)', row.username)
    return a.group(0)[2:-1]

In [10]:
meta_data["username"] = meta_data.apply(getUsername,axis=1)

In [11]:
meta_data = meta_data[(meta_data['images'].notnull())]
meta_data.head(2)

,caption,comments,image_urls,images,likes,pro,user_url,username
24,{ Timeless Beauty } • Breathtaking place to fe...,196.0,http://scontent.cdninstagram.com/t51.2885-15/s...,[{'url': 'http://scontent.cdninstagram.com/t51...,25518.0,1,http://www.imgrum.org/user/brahmino/1918184,brahmino
25,{ Golden Morning } • Happy to wake up early an...,298.0,http://scontent.cdninstagram.com/t51.2885-15/s...,[{'url': 'http://scontent.cdninstagram.com/t51...,28436.0,1,http://www.imgrum.org/user/brahmino/1918184,brahmino


### Print URLS to get follower data

In [12]:
unames = list(set(meta_data['username']))

In [13]:
urls = []
for i in range(len(unames)):
    urls.append('https://www.instagram.com/'+unames[i]+'/?__a=1')
urls = pd.DataFrame(urls)

In [14]:
urls.to_csv(scraping_path+'user_urls.txt', index=False)

In [15]:
len(meta_data)

160418

In [16]:
followers = pd.read_csv(data_path+'followers_data.csv')
followers = followers.drop(['images'],1)

In [17]:
followers.head()

,insta_id,is_verified,following,full_name,followers,posts,username
0,208131708,False,356,▪️5ft. dad ▪️,4056,69,airinyung
1,1579219460,False,232,Comedy Ahmed,1012,1357,comedyahmed
2,264364242,False,483,Jesse Herzog,228722,1782,jesseherzog
3,454515755,False,561,pipinya_zizi,1187,565,pipinya_zizi
4,1462059659,False,2602,MO 👼🏾,20351,807,phobymo


In [18]:
meta_data = pd.merge(meta_data,followers, on = 'username').reset_index()

In [19]:
meta_data.head(2)

,index,caption,comments,image_urls,images,likes,pro,user_url,username,insta_id,is_verified,following,full_name,followers,posts
0,0,{ Timeless Beauty } • Breathtaking place to fe...,196.0,http://scontent.cdninstagram.com/t51.2885-15/s...,[{'url': 'http://scontent.cdninstagram.com/t51...,25518.0,1,http://www.imgrum.org/user/brahmino/1918184,brahmino,1918184,False,341,Simone Bramante,830765,1194
1,1,{ Golden Morning } • Happy to wake up early an...,298.0,http://scontent.cdninstagram.com/t51.2885-15/s...,[{'url': 'http://scontent.cdninstagram.com/t51...,28436.0,1,http://www.imgrum.org/user/brahmino/1918184,brahmino,1918184,False,341,Simone Bramante,830765,1194


In [20]:
print(type(meta_data.images[0][1:-1]))
print((meta_data.images[0]))

<class 'str'>
[{'url': 'http://scontent.cdninstagram.com/t51.2885-15/s480x480/e35/c0.134.1080.1080/17125902_1260649764012807_8695089225495740416_n.jpg?ig_cache_key=MTQ2NTI5MDU2OTYzMDAxMTg1OQ%3D%3D.2.c', 'path': 'full/2299079fa299a4b2ad784ebe588243865f45791c.jpg', 'checksum': '6d382a79e19ef8eef973b715d63180aa'}]


In [21]:
def makeJson(row):
    row = row[1:-1]
    json_acceptable_string = row.replace("'", "\"")
    return json.loads(json_acceptable_string)

In [22]:
meta_data["images"] = meta_data.images.apply(makeJson)
print(type(meta_data.images[0]))
meta_data.head(2)

<class 'dict'>


,index,caption,comments,image_urls,images,likes,pro,user_url,username,insta_id,is_verified,following,full_name,followers,posts
0,0,{ Timeless Beauty } • Breathtaking place to fe...,196.0,http://scontent.cdninstagram.com/t51.2885-15/s...,{'path': 'full/2299079fa299a4b2ad784ebe5882438...,25518.0,1,http://www.imgrum.org/user/brahmino/1918184,brahmino,1918184,False,341,Simone Bramante,830765,1194
1,1,{ Golden Morning } • Happy to wake up early an...,298.0,http://scontent.cdninstagram.com/t51.2885-15/s...,{'path': 'full/d92fc8d70b8e71fa9ea53ced58dcd2d...,28436.0,1,http://www.imgrum.org/user/brahmino/1918184,brahmino,1918184,False,341,Simone Bramante,830765,1194


In [23]:
pic_path = []
for i in range(len(meta_data)):
    pic_path.append(meta_data['images'][i]['path'][5:])
meta_data['pic_path'] = pic_path

checksum = []
for i in range(len(meta_data)):
    checksum.append(meta_data['images'][i]['checksum'])
meta_data['checksum'] = checksum 

In [24]:
meta_data = meta_data.drop(['images','index'],1)
meta_data.head(2)

,caption,comments,image_urls,likes,pro,user_url,username,insta_id,is_verified,following,full_name,followers,posts,pic_path,checksum
0,{ Timeless Beauty } • Breathtaking place to fe...,196.0,http://scontent.cdninstagram.com/t51.2885-15/s...,25518.0,1,http://www.imgrum.org/user/brahmino/1918184,brahmino,1918184,False,341,Simone Bramante,830765,1194,2299079fa299a4b2ad784ebe588243865f45791c.jpg,6d382a79e19ef8eef973b715d63180aa
1,{ Golden Morning } • Happy to wake up early an...,298.0,http://scontent.cdninstagram.com/t51.2885-15/s...,28436.0,1,http://www.imgrum.org/user/brahmino/1918184,brahmino,1918184,False,341,Simone Bramante,830765,1194,d92fc8d70b8e71fa9ea53ced58dcd2db3cbe95db.jpg,3757b15a6fe424b2c543f07e48d87d60


In [25]:
meta_data.to_csv(data_path+'meta_data.csv', index = False)

In [26]:
with open(data_path+'meta_data.pkl', 'wb') as picklefile:
    pickle.dump(meta_data, picklefile)